# Getting Data from the Internet

There is a ton of good data on the internet, but it can be hard to access.  In this lesson we will learn just enough about web scraping to... get in trouble.  

**Important**: stay out of trouble!

### Best Practices

1. Don't break anything.  Many rapid requests to smaller sites can overload the host server.
2. Use a published API if possible - it is more robust and usually much easier!
3. Respect the policy published at `robots.txt` 
4. Don't spoof your UserAgent (or try to trick the server into thinking you are a person)
5. Read the Terms of Service for the site and follow it.

# Requests

`requests` is a python package that allows you to use Python to interact with the internet!  There are other packages, but I find `requests` to be much easier to use.

In fact, to get the UCSD home page is a simple as
```
import requests
text = requests.get("https://ucsd.edu").text
```
But before we do that, we need to learn just a little bit more.

# Status Codes

When we request data from a website, the server responds with a HTTP status code.  The most common response is `200` which means things went well.  Other times you will get a different status code saying something else happened - you might be familiar with a `404` which means the page wasn't found.

This great site lists http status codes: [https://httpstat.us/](https://httpstat.us/).

But better yet, it has example sites that return a certain code, so you can test!  So, for example, https://httpstat.us/404 returns a `404`

In [1]:
import requests

r = requests.get("https://httpstat.us/404")
print(r.status_code)

404


In [2]:
r = requests.get("https://httpstat.us/404")
r.status_code
r.text

'404 Not Found'

You can check if the call went ok with `r.ok` which returns a boolean.

After you run the code below, read up on each of the status codes at [https://httpstat.us/](https://httpstat.us/).

In [3]:
statusCodes = [200, 404, 403, 429]

for statusCode in statusCodes:
    r = requests.get("https://httpstat.us/" + str(statusCode))
    print(str(statusCode) + " ok: " + str(r.ok))

200 ok: True
404 ok: False
403 ok: False
429 ok: False


In [5]:
# Or raise an exception when there is a not-ok status code

r = requests.get("https://httpstat.us/404")
r.raise_for_status()

HTTPError: 404 Client Error: Not Found for url: https://httpstat.us/404

# Robots.txt

Many sites have a published policy allowing or disallowing automatic access to their site.  They may also specify which user-agent is allowed to automatically access specific part of the site. They use a text file `robots.txt` and you can learn more about it [here](https://moz.com/learn/seo/robotstxt).

The code below checks if the `robot.txt` file prohibits you from scraping the site.  Remember the best practices above - just because you aren't prohibited by the robots policy doesn't mean you can scrape the site!

In [6]:
from urllib.parse import urlparse
import urllib.robotparser

# This code checks the robots.txt file
def canFetch(url):

    parsed_uri = urlparse(url)
    domain = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)

    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(domain + "/robots.txt")
    try:
        rp.read()
        canFetchBool = rp.can_fetch("*", url)
    except:
        canFetchBool = None
    
    return canFetchBool

In [7]:
url = "http://blink.ucsd.edu/search"
canFetch(url)

False

In [8]:
url = "https://datascience.ucsd.edu/academics/undergraduate/"
canFetch(url)

True

# Getting the HTML

Now we can request a website!  Let's see what is on the UCSD Data Science Events page.

In [9]:
url = "https://datascience.ucsd.edu/academics/undergraduate/"

r = requests.get(url)
    
urlText = r.text

Nchars = 10000
print(urlText[:Nchars]) # Print the first 500 characters
print("\n\n... " + str(len(urlText)-Nchars) + " additional characters")


<!DOCTYPE html>
<html lang="en-US"><head>
    <meta charset="UTF-8">
		            <meta name="title" content="">
            <meta name="keywords" content="">
          	<meta name="description" content="">
                <link rel="profile" href="http://gmpg.org/xfn/11">
    <link rel="pingback" href="https://datascience.ucsd.edu/xmlrpc.php">    
                <link rel="shortcut icon" href="https://devdatasucsand.com/wp-content/uploads/2019/02/Single-Icon-white-PNG.png">
            <title>Undergraduate &#8211; Halıcıoğlu Data Science Institute</title>
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0"><link rel='dns-prefetch' href='//fonts.googleapis.com' />
<link rel='dns-prefetch' href='//s.w.org' />
<link rel="alternate" type="application/rss+xml" title="Halıcıoğlu Data Science Institute &raquo; Feed" href="https://datascience.ucsd.edu/feed/" />
<link rel="alternate" type="application/rss+xml" title="Halıcıoğlu Data Scien

In [10]:
len(r.text)

105052

# Cleaning

Wow, that is gross looking!  It is raw HTML, which the browser uses to make the viewable site.  To process it we can use [Beautiful Soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

**Warning** BeautifulSoup has changed quite a bit between versions, so make sure you are looking at documentation for the version you are using (4 here).
Let's follow this example on using BeautifulSoup: ([example](https://codeburst.io/web-scraping-101-with-python-beautiful-soup-bb617be1f486))

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(urlText, 'html.parser')

In [12]:
# let's check once more if it is safe and legal to scrape from.
canFetch(url)

True

In [13]:
page_response = requests.get(url,timeout=5)
# here, we fetch the content from the url, using the requests library

In [14]:
page_content = BeautifulSoup(page_response.content, "html.parser")
#we use the html parser to parse the url content and store it in a variable.

paragraphs = page_content.find_all("p")
# Here we get all content within <p> paragraphs

In [14]:
# let's see it one by one:
for paragraph in paragraphs:
    print(paragraph)

<p><span style="letter-spacing: 0.2px;">The goal of these programs is to train a generation of students who are equally versed in predictive modeling, data analysis and computational techniques.</span></p>
<p>The undergraduate major in Data Science will prepare our students for a career in the data-centric society and to advance knowledge in the emerging field of Data Science.</p>
<p>The major consists of 116 units in addition to general education requirements, some of which may be covered by the major. The required courses include courses in mathematics (especially linear algebra and probability), computer science (programming, data structures and abstractions, data mining), and statistics (estimation, testing, and exploratory data analysis).</p>
<p>A 12-unit lower division course sequence in physics, chemistry or biology will strengthen background in natural sciences. The program includes 20 units of elective courses that enable students to embark upon an in-depth exploration of one 

In [15]:
#let's get rid of all the html code:
for paragraph in paragraphs:
    print(paragraph.text)

The goal of these programs is to train a generation of students who are equally versed in predictive modeling, data analysis and computational techniques.
The undergraduate major in Data Science will prepare our students for a career in the data-centric society and to advance knowledge in the emerging field of Data Science.
The major consists of 116 units in addition to general education requirements, some of which may be covered by the major. The required courses include courses in mathematics (especially linear algebra and probability), computer science (programming, data structures and abstractions, data mining), and statistics (estimation, testing, and exploratory data analysis).
A 12-unit lower division course sequence in physics, chemistry or biology will strengthen background in natural sciences. The program includes 20 units of elective courses that enable students to embark upon an in-depth exploration of one or more areas in which Data Science can profitably be applied. Alter

# Next steps

From here you can do a number of different things!

* Choose a website that you like! Check if you can web scape it!
* Get the text
* Pull text down and use NLP from last week (like sentiment analysis)
* Monitor a site daily for changes.
* Use the text to create your own search engine!

In [16]:
# your code here!